<a href="https://colab.research.google.com/github/Mohamed28112003/Muffakir/blob/main/Notebooks/Frozen_RAG_%26_HyDE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Libraries**

In [ ]:
!pip install  transformers
!pip install langchain
!pip install -U langchain-community
!pip install unstructured
!pip install "unstructured[pdf]"
!apt-get install -y poppler-utils
!pip install LangChain -q
!pip install langchain transformers
!pip install chromadb
!pip install sentence-transformers
!pip install rank-bm25 sentence-transformers
!pip uninstall nltk -y
!pip install nltk
!pip install openai
!pip install groq
!pip install langchain-groq
!pip install gradio

In [ ]:
from transformers import AutoTokenizer
from langchain.text_splitter import CharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import PromptTemplate
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
import re
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
import chromadb
import os
from langchain_groq import ChatGroq
from langchain.schema import HumanMessage
import gradio as gr
from langchain.prompts import ChatPromptTemplate
from langchain.schema import Document


## **Load TXT**

In [ ]:
data_loader = DirectoryLoader("/content/Data",
                             glob="*.txt",
                             show_progress=True)


In [ ]:
text_splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0,
    separators=["Page number"]
)

In [ ]:
data = data_loader.load_and_split(text_splitter=text_splitter_recursive) # Load documents

100%|██████████| 10/10 [02:18<00:00, 13.83s/it]


In [ ]:
print(f"Documents : {len(data)}.")

Documents : 3391.


##  **Clean Text**

In [ ]:
data[9].page_content

In [ ]:

def clean_arabic_text(text):
    text = re.sub(r'Page number: \d+', '', text)
    text = re.sub(r'- \d+ -', '', text)
    text = re.sub(r'\b\d+\b', '', text)
    text = re.sub(r'[^\u0600-\u06FF\s]', '', text)
    text = re.sub(r'[^\w\s\u0600-\u06FF]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

for i in range(len(data)):
    data[i].page_content = clean_arabic_text(data[i].page_content)


In [ ]:
data[9].page_content

## **Embeddding**

In [ ]:
base_model = SentenceTransformer('intfloat/multilingual-e5-large')

In [ ]:



path = "/content/DB"
client = chromadb.PersistentClient(path=path)

db_file = client.get_or_create_collection(name='Book',
                                          metadata={"hnsw:space": "cosine"})

In [ ]:
def embed(texts):
    return base_model.encode(texts)

i = 0
for chunk in data:
    embeddings = embed([chunk.page_content])
    db_file.add(
        documents=[chunk.page_content],
        metadatas=[chunk.metadata],
        ids=[f"chunk_{i}"],
        embeddings=embeddings  # Pass the embeddings directly
    )
    i += 1

In [ ]:
db_file.get(ids=["chunk_2"])

In [ ]:
def retrieve_documents_embeddings(query_embedding, k=10):
    query_embedding_list = query_embedding.tolist()

    results = db_file.query(
        query_embeddings=[query_embedding_list],

        n_results=k)
    return results

## **LLM**

In [ ]:


os.environ["GROQ_API_KEY"] = "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"

llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model="llama-3.1-70b-versatile",
    temperature=0.7,
    max_tokens=500,
)

messages = [
    HumanMessage(content="Hello")
]

response = llm.invoke(messages)

for chunk in response:
    content = chunk[0] if isinstance(chunk, tuple) else chunk
    print(content or "", end="")


contentadditional_kwargsresponse_metadatatypenameidexampletool_callsinvalid_tool_callsusage_metadata

In [ ]:
print(response.content)

Hello. It's nice to meet you. Is there something I can help you with or would you like to chat?


In [ ]:
#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

## **Frozen RAG**

In [ ]:
qna_template = "\n".join([
    "Act as a knowledgeable law professor. Analyze the provided legal context and respond to the subsequent question with thoroughness and clarity. If the information needed to answer the question is not present in the context, respond with 'NO ANSWER IS AVAILABLE.'",
    "give the answer in details with Arabic accent ",

    "### Context",
    "{context}",

    "### Question",
    "{question}",

    "### Answer:",
])

qna_prompt = PromptTemplate(
    template=qna_template,
    input_variables=['context', 'question'],
    verbose=True
)

stuff_chain = load_qa_chain(llm, chain_type="stuff", prompt=qna_prompt)


In [ ]:
# Sample query
query = ''''

ما أهمية التنفيذ الجبري في قانون المرافعات؟

'''

query_embedding = base_model.encode(query)

similar_documents = retrieve_documents_embeddings(query_embedding, k=7)



In [ ]:
documents = similar_documents['documents'][0]
metadatas = similar_documents['metadatas'][0]

from langchain.schema import Document
formatted_documents = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(documents, metadatas)
]

In [ ]:
answer = stuff_chain(
    {
        "input_documents": formatted_documents,
        "question": query
    },
    return_only_outputs=True,
)



In [ ]:
output_text = answer['output_text']

print("Generated Answer:")
print("=" * 50)
print(output_text)


### **Gradio Frozen RAG**

In [ ]:

def generate_answer(query):
    query_embedding = base_model.encode(query)

    similar_documents = retrieve_documents_embeddings(query_embedding, k=7)

    documents = similar_documents['documents'][0]
    metadatas = similar_documents['metadatas'][0]

    formatted_documents = [
        Document(page_content=doc, metadata=meta)
        for doc, meta in zip(documents, metadatas)
    ]
    answer = stuff_chain(
        {
            "input_documents": formatted_documents,
            "question": query
        },
        return_only_outputs=True,
    )

    output_text = answer['output_text']

    similar_docs_str = "\n".join([f"**Document {i+1}:**\n{doc}\n**Metadata:** {meta}" for i, (doc, meta) in enumerate(zip(documents, metadatas))])

    return output_text, similar_docs_str



iface = gr.Interface(
    fn=generate_answer,
    inputs=gr.Textbox(lines=5, placeholder="Enter your query here..."),
    outputs=[
        gr.Textbox(label="Answer"),
        gr.Textbox(label="Similar Documents with Metadata")
    ],

        examples=[
        "ما أهمية التنفيذ الجبري في قانون المرافعات؟",
        "ما هي قواعد الاختصاص القضائي في القانون المصري؟",
    ],
    title="Muffakir",
    description="Ask your questions"
)

iface.launch(share=True)

## **Hypothetical Document Embeddings**


In [ ]:
query_hyde = ''''

ما أهمية التنفيذ الجبري في قانون المرافعات؟

'''

In [ ]:

template_hyde = "\n".join([
    "Act as a Law professor from egypt. Analyze the provided law question and respond to the subsequent question with thoroughness and clarity.",
    "give the answer in details in arabic only ",


    "### Question",
    "{question}",

    "### Answer:",
])
prompt_hyde = ChatPromptTemplate.from_template(template_hyde)

hypothetical_document = llm.invoke(prompt_hyde.format(question=query_hyde))


In [ ]:


hypothetical_document_embedding = base_model.encode(hypothetical_document.content)

similar_documents_hyde = retrieve_documents_embeddings(hypothetical_document_embedding, k=5)



documents_hyde = similar_documents_hyde['documents'][0]
metadatas_hyde = similar_documents_hyde['metadatas'][0]

from langchain.schema import Document
formatted_documents_hyde = [
    Document(page_content=doc, metadata=meta)
    for doc, meta in zip(documents_hyde, metadatas_hyde)
]


In [ ]:
answer = stuff_chain(
        {
            "input_documents": formatted_documents_hyde ,
            "question": query
        },
        return_only_outputs=True
    )


output_text = answer['output_text']

# Print the formatted answer
print("Generated Answer:")
print("=" * 50)
print(output_text)


### **Gradio HyDE**

In [ ]:

template_hyde = "\n".join([
    "Act as a Law professor from Egypt. Analyze the provided law question and respond to the subsequent question with thoroughness and clarity.",
    "give the answer in details in arabic only.",

    "### Question",
    "{question}",

    "### Answer:",
])
prompt_hyde = ChatPromptTemplate.from_template(template_hyde)

def law_chatbot(query):
    try:
        hypothetical_document = llm.invoke(prompt_hyde.format(question=query))

        hypothetical_document_embedding = base_model.encode(hypothetical_document.content)

        similar_documents_hyde = retrieve_documents_embeddings(hypothetical_document_embedding, k=5)

        documents_hyde = similar_documents_hyde['documents'][0]
        metadatas_hyde = similar_documents_hyde['metadatas'][0]

        formatted_documents_hyde = [
            Document(page_content=doc, metadata=meta)
            for doc, meta in zip(documents_hyde, metadatas_hyde)
        ]

        answer = stuff_chain(
            {
                "input_documents": formatted_documents_hyde,
                "question": query
            },
            return_only_outputs=True
        )

        output_text = answer['output_text']

        documents_display = "\n\n".join([f"Document {i+1}:\n{doc}" for i, doc in enumerate(documents_hyde)])
        metadata_display = "\n\n".join([f"Metadata {i+1}:\n{meta}" for i, meta in enumerate(metadatas_hyde)])

        return output_text, documents_display, metadata_display

    except Exception as e:
        return f"An error occurred: {str(e)}", "", ""

iface = gr.Interface(
    fn=law_chatbot,
    inputs="text",
    outputs=[
        gr.Textbox(label="Chatbot Answer", lines=10),
        gr.Textbox(label="Similar Documents", lines=10),
        gr.Textbox(label="Metadata", lines=10),
    ],


        examples=[
        "ما أهمية التنفيذ الجبري في قانون المرافعات؟",
        "ما هي قواعد الاختصاص القضائي في القانون المصري؟",
    ],

    title="Muffakir HyDE ",
    description="Ask any question",

)

iface.launch()
